In [90]:
from data_loader import data_reader, colnames, colnames_bid
from matplotlib import pyplot as plt
import math
import pandas as pd

In [91]:
#DONT FORGET TO LOAD THAT OTHER FUCKING DATASET, I KNOW YOU ARE LAZY
# BUT JUST DONT FORGET IT
# YEAH, SURE, NO PROBLEM
# YOU CAN DO IT LATER
# JUST DONT FORGET IT

imp = data_reader('data\\imp.20131020.txt', colnames, 
                verbose=False, fuck_parsing=True)
bid = data_reader('data\\bid.20131020.txt', colnames_bid, verbose=False, fuck_parsing=True)
clk = data_reader('data\\clk.20131020.txt', colnames, verbose=False, fuck_parsing=True)
conv = data_reader('data\\conv.20131020.txt', colnames, verbose=False, fuck_parsing=True)


In [106]:
bid_rel = bid[['bid_id', 'timestamp']]
imp_rel = imp[['bid_id', 'paying_price']]
# clk_rel = clk[['bid_id', 'paying_price']]
# conv_rel = conv[['bid_id', 'paying_price']]
bids = bid_rel.merge(imp_rel,how='left', on='bid_id').sort_values('timestamp')
bids['not_bought'] = [math.isnan(price) for price in bids.paying_price]
bids.paying_price = bids.paying_price.fillna(0)

In [108]:
G = 1e5 / 2
B = 6e6 / 2
target_price =  B/G

def split_explore_exploit(bids, share_exploration):
    N = bids.shape[0]
    this_many = int(share_exploration * N)
    bids_explore = bids.iloc[:this_many, :]
    bids_exploit = bids.iloc[this_many:, :]
    return bids_explore, bids_exploit, N

In [118]:
share_exploration = 0.01
bids_explore, bids_exploit, N = split_explore_exploit(bids, share_exploration)

print(bids_explore.shape)
print(bids_exploit.shape)

def explore(bids_explore, target_price, N, rounds=10):
    bid_price = target_price
    N_expl = bids_explore.shape[0]
    step = int(N_expl / rounds)
    start = 0
    stop = step
    for round in range(rounds):
        won = 0
        cost = 0
        for index, bid in bids_explore \
                            .iloc[start:stop, :] \
                            .iterrows():
            if bid_price > bid.paying_price:
                won += 1
                cost += bid.paying_price
        winrate = won / step
        price = cost / won
        exp_views = N * winrate
        exp_price = cost / step
        
        if  (exp_views < G) or (exp_price < target_price):
            bid_price = bid_price * 2
            print('doubling')
        else:
            print('breaking in round %s' % round)
            print('E(views): %s > target_views %s' % (exp_views, G))
            print('E(price): %.2f > target_price %s' % (exp_price, target_price))
            break
        start += step
        stop += step
    return bid_price
explore(bids_explore, target_price, N)


(3271, 4)
(323832, 4)
doubling
doubling
doubling
doubling
doubling
doubling
doubling
breaking in round 7
E(views): 327103.0 > target_views 50000.0
E(price): 61.14 > target_price 60.0


7680.0

(327103, 4)